# 并发

## 全局解释锁 (GIL)

CPython 解释器本身不是线程安全的，因此有全局解释器锁 (GIL) ，一次只允许使用一个线程执行 Python 字节码。因此，一个 Python 进程不能同时使用多个 CPU 。

### I/O 密集型操作

标准库中所有执行阻塞型 I/O 操作的函数，在等待操作系统返回结果时都会释放 GIL 。这意味着在 Python 语言这个层次上可以使用多线程，I/O  密集型的程序能从中受益。(`time.sleep()` 函数也会释放 GIL)

### CPU 密集型操作

使用 ProcessPoolExecutor 类把工作分配给多个进程处理可以实现真正的并行运算。因此，如果需要做 CPU 密集型处理，可以使用它绕开 GIL ，从而利用所有可用的 CPU 。

## concurrent.futures 模块

concurrent.futures 模块的主要特色是 ThreadPoolExecutor 和 ProcessPoolExecutor 类，这两个类实现的接口能分别在不同的线程或进程中执行可调用的对象。

通常情况下 future 对象不应由用户创建，而是由并发框架 (concurrent.futures 或 asyncio) 来实例化。

### `Executor.map(func, *iterables, timeout=None, chunksize=1)`

chunksize 只对 ProcessPoolExecutor 有用，用于切分 iterables ，提高运行效率。因为 future 结果的返回涉及到 IPC ，如果每个进程每次消耗 iterables 中的一个数据，整个过程涉及多个 IPC ，这样效率不高；但如果对 iterables 进行切分，N 个 数据同时交给一个进程进行处理，运算结果通过一个 IPC 一并返回，这样就可以提升效率。

返回值是一个迭代器，**迭代器的 `__next__` 方法调用各个 future 对象的 `result` 方法，得到各个 future 的结果。**


In [1]:
from concurrent.futures import ThreadPoolExecutor
import time

def sleep_and_double(value):
    time.sleep(value)
    return value * 2

time0 =time.time()

with ThreadPoolExecutor(max_workers=4) as executor:
    fs = executor.map(sleep_and_double, [3, 2, 1])
    time_delta = time.time() - time0
    print("time consumed: ", time_delta)
fs

time consumed:  0.0017189979553222656


<generator object Executor.map.<locals>.result_iterator at 0x110287c50>

In [2]:
[v for v in fs]

[6, 4, 2]

### `Executor.sumit(fn, *args, **kwargs)`

In [3]:
time0 = time.time()
with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(sleep_and_double, 3)
    time_delta = time.time() - time0
    print("time consumed: ", time_delta)

future.result()

time consumed:  0.0014939308166503906


6

### `concurrent.futures.as_completed(fs, timeout=None)`